# 소스코드를 통한 예측 결과를 넣지 않은 상태입니다. 후에 추가 예정입니다.

#필요한 파일들 : 인증키 파일, pickle 파일, 추출된 압축파일

# 메타데이터 파일 파싱 후 리스트 형태로 저장

##메타데이터 전처리 및 추가

###모든 소스코드 추출

In [ ]:
import os
import csv
import re
import zipfile

def remove_comments(code):
    # 여러 줄 주석 제거
    code = re.sub(r"'''(.*?)'''", '', code, flags=re.DOTALL)
    code = re.sub(r'"""(.*?)"""', '', code, flags=re.DOTALL)
    # 한 줄 주석 제거
    code = re.sub(r'#.*', '', code)
    return code

def process_directory(root_path):
    rows = []

    # root_path 내부의 모든 하위 디렉터리 탐색
    for dir_name in os.listdir(root_path):
        dir_path = os.path.join(root_path, dir_name)
        if os.path.isdir(dir_path):
            merged_code = ''
            for root, _, files in os.walk(dir_path):
                for file in files:
                    if file.endswith('.py'):
                        file_path = os.path.join(root, file)
                        try:
                            with open(file_path, 'r', encoding='utf-8') as f:
                                raw_code = f.read()
                                cleaned_code = remove_comments(raw_code)
                                merged_code += cleaned_code + '\n'
                        except Exception as e:
                            print(f"⚠️ {file_path} 읽기 실패: {e}")
            if merged_code.strip():
                rows.append([dir_name, merged_code.strip()])
    return rows

def save_to_csv(data, output_file='merged_by_directory.csv'):
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Directory', 'MergedCodeWithoutComments'])
        writer.writerows(data)

zip_dir = "/content/python-packages-1757531529324.zip"
extract_dir = "/content/extracted_files"

# 압축 해제
with zipfile.ZipFile(zip_dir, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)


# 사용 예시
root_path = '/content/extracted_files/source'  # 여기에 네가 지정한 경로 입력
data = process_directory(root_path)
save_to_csv(data)
print(f"✅ CSV 저장 완료: {len(data)}개 디렉터리 처리됨")


✅ CSV 저장 완료: 2개 디렉터리 처리됨


###나머지 전처리

In [ ]:
import os
import re
import pandas as pd
import zipfile

# 이름과 이메일 파싱 함수
def parse_name_email(text):
    match = re.match(r"(.*)<(.*@.*)>", text)
    if match:
        name = match.group(1).strip()
        email = match.group(2).strip()
        return name, email
    return None, None

# 메타데이터 파싱 함수
def parse_metadata(file_path):
    target_keys = {
        "name", "summary","author", "author-email",  "version",
        "maintainer", "maintainer-email"
    }
    metadata = {}

    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            if ':' in line:
                key, value = map(str.strip, line.split(':', 1))

                key_lower = key.lower()

                if key_lower in target_keys:
                    metadata[key_lower] = value



    # author가 없거나 값이 비어 있을 경우
    if not metadata.get("author"):
        if metadata.get("author-email"):
            name, email = parse_name_email(metadata["author-email"])
            if name and email:
                metadata["author"] = name
                metadata["author-email"] = email

    # author_email이 없거나 값이 비어 있을 경우
    if not metadata.get("author-email"):
        if "maintainer_email" in metadata and metadata["maintainer-email"].strip():
            metadata["author-email"] = metadata["maintainer-email"]

    # author가 없거나 값이 비어 있을 경우 → maintainer로 대체
    if not metadata.get("author"):
        if "maintainer" in metadata and metadata["maintainer"].strip():
            metadata["author"] = metadata["maintainer"]

    return metadata


metadata_dir = os.path.join(extract_dir, "metadata")
meta_datas = []

for file in os.listdir(metadata_dir):
    if file.endswith(".txt"):
        metadata_path = os.path.join(metadata_dir, file)
        metadata = parse_metadata(metadata_path)
        meta_datas.append(metadata)


In [ ]:
!pip install google-cloud-bigquery

전처리 1

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("/content/plated-mantis-471407-m4-b14f1b3e761d.json")
client = bigquery.Client(credentials=credentials, project="plated-mantis-471407-m4")


In [ ]:
import requests
from google.api_core import exceptions as gcp_exceptions
#다운로드 받기 방법 1
def get_pepy_downloads(package_name, api_key):
    """
    PePy.tech API를 사용하여 특정 Python 패키지의 총 다운로드 수를 반환합니다.

    Parameters:
        package_name (str): 조회할 PyPI 패키지 이름
        api_key (str): PePy API 키 (https://pepy.tech에서 발급 가능)

    Returns:
        int: 총 다운로드 수 (실패 시 -1 반환)
    """
    url = f"https://api.pepy.tech/api/v2/projects/{package_name}"
    headers = {
        "X-API-Key": api_key
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            return data.get("total_downloads", -1)
        else:
            #print(f"API 요청 실패: {response.status_code}")
            return -1
    except Exception as e:
        #print(f"오류 발생: {e}")
        return -1






def get_download_count_bq(package_name: str, service_account_json: str | None = None) -> int:
    """
    bigquery-public-data.pypi.file_downloads에서 해당 패키지의 다운로드 수를 계산합니다.
    service_account_json을 지정하면 그 키로 클라이언트를 만듭니다.
    실패 시 -1 반환.
    """
    try:
        if service_account_json:
            client = bigquery.Client.from_service_account_json(service_account_json)
        else:
            client = bigquery.Client()  # ADC 또는 환경변수 기반 인증 사용

        query = """
        SELECT COUNT(*) AS total_downloads
        FROM `bigquery-public-data.pypi.file_downloads`
        WHERE file.project = @pkg
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("pkg", "STRING", package_name)
            ]
        )

        query_job = client.query(query, job_config=job_config)
        result = query_job.result()  # 블로킹: 결과 받을 때까지 기다림

        for row in result:
            # row["total_downloads"] 또는 row.total_downloads 모두 가능
            return int(row["total_downloads"])

        return 0
    except gcp_exceptions.GoogleAPICallError as e:
        #print("BigQuery API 호출 오류:", e)
        return -1
    except Exception as e:
        #print("BigQuery 조회 중 오류:", e)
        return -1

def download_unified(package_name):
    download_count = get_pepy_downloads(package_name,"0SRbc/jRFsHYxOShwIQ/N0jtrKf1syMW")
    if(download_count==-1):
      download_count = get_download_count_bq(package_name, "/content/plated-mantis-471407-m4-b14f1b3e761d.json")

    return download_count

for meta_data in meta_datas:
    package_name = meta_data.get("name")
    if package_name:
        download_count = download_unified(package_name)
        meta_data["download"] = download_count



전처리 2

In [ ]:
import pandas as pd
import numpy as np
import math
import re
from collections import Counter
from sklearn.preprocessing import StandardScaler

# 1️⃣ 리스트 → DataFrame 변환
df = pd.DataFrame(meta_datas)

# 2️⃣ 다운로드 수 정규화 (로그 변환)
df["download"] = df["download"].fillna(0).astype(int)
df["download_log"] = df["download"].apply(lambda x: np.log1p(x))

scaler = StandardScaler()
df["download_scaled"] = scaler.fit_transform(df[["download_log"]])

# 3️⃣ 설명 길이 분석
df["summary"] = df["summary"].fillna("")
df["summary_length"] = df["summary"].apply(len)
df["summary_too_short"] = df["summary_length"] < 10
df["summary_too_long"] = df["summary_length"] > 300



# 4️⃣ 설명 자동 생성 여부 추정 (엔트로피 기반)
def shannon_entropy(s):
    if not s:
        return 0
    prob = [v / len(s) for v in Counter(s).values()]
    return -sum(p * math.log2(p) for p in prob)

df["summary_entropy"] = df["summary"].apply(shannon_entropy)
df["summary_low_entropy"] = df["summary_entropy"] < 3.5

# 5️⃣ 버전 형식 검증
def is_valid_version(v):
    return bool(re.match(r"^\d+\.\d+\.\d+$", str(v).strip()))

df["version_valid"] = df["version"].apply(is_valid_version)


전처리 3

In [ ]:
!pip install Levenshtein

In [ ]:
import pandas as pd
import requests
import re
from Levenshtein import distance as levenshtein_distance

# 1️⃣ PyPI 상위 패키지 목록 가져오기
def get_pypi_top_packages():
    url = "https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json"
    response = requests.get(url)
    data = response.json()
    return [pkg['project'] for pkg in data['rows']]

# 2️⃣ 핵심 단어 추출 함수 (접두/접미어 제거)
def extract_core_name(name):
    return re.split(r"[-_.]", name.lower())[0]

# 3️⃣ 오타 기반 유사성 판별 함수
def is_typo_like(pkg_name, legit_list):
    name = extract_core_name(pkg_name)
    for legit in legit_list:
        legit_core = extract_core_name(legit)
        if levenshtein_distance(name, legit_core) == 1 and abs(len(name) - len(legit_core)) <= 1:
            return True
    return False

# 4️⃣ 인기 패키지 목록 불러오기
pypi_packages = get_pypi_top_packages()

# 6️⃣ 오타 기반 악성 탐지 피처 생성

df["is_typo_like"] = df["name"].apply(lambda x: is_typo_like(x, pypi_packages))

In [ ]:
# 누락된 피처 생성
df["summary_too_short"] = df["summary_length"] < 10
df["summary_too_long"] = df["summary_length"] > 300
df["summary_low_entropy"] = df["summary_entropy"] < 3.5

df["download_too_low"] = df["download_log"] < df["download_log"].quantile(0.05)
df["download_too_high"] = df["download_log"] > df["download_log"].quantile(0.95)

df["is_disposable"] = False


In [ ]:
# 결측치가 있을 경우 -1로 대체 (이미 처리된 상태라면 생략 가능)
df.fillna(0, inplace=True)

# vulnerability_status와 cwe_label을 기반으로 새로운 피처 생성
def combined_threat(row):
    if row["vulnerability_status"] == 1 and row["cwe_label"] == 1:
        return 2
    elif row["vulnerability_status"] == 1 or row["cwe_label"] == 1:
        return 1
    else:
        return 0

df["threat_level"] = df.apply(combined_threat, axis=1)

# 3. download_log 스케일링 (0~1 범위로 정규화)
scaler = MinMaxScaler()
df["download_log_scaled"] = 1-scaler.fit_transform(df[["download_log"]])



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pkg_resources

# Generate requirements.txt
libraries = [
    "pandas",
    "numpy",
    "math", # math is a built-in module, no version needed
    "re", # re is a built-in module, no version needed
    "collections", # collections is a built-in module, no version needed
    "sklearn", # Use scikit-learn for requirements.txt
    "requests",
    "Levenshtein",
    "google-cloud-bigquery",
    "google-api-core",
    "zipfile", # zipfile is a built-in module, no version needed
    "os", # os is a built-in module, no version needed
    "csv", # csv is a built-in module, no version needed
    "pickle", # pickle is a built-in module, no version needed
    "xgboost" # Assuming xgboost was used for the model
]

with open("requirements.txt", "w") as f:
    for lib in libraries:
        try:
            # For built-in modules, skip writing version
            if lib in ["math", "re", "collections", "zipfile", "os", "csv", "pickle"]:
                f.write(f"{lib}\n")
            elif lib == "sklearn": # Handle scikit-learn
                 f.write(f"scikit-learn=={pkg_resources.get_distribution('scikit-learn').version}\n")
            else:
                f.write(f"{lib}=={pkg_resources.get_distribution(lib).version}\n")
        except pkg_resources.DistributionNotFound:
            f.write(f"{lib} # Version not found\n")


print("✅ requirements.txt 파일이 생성되었습니다.")

/tmp/ipython-input-3137591171.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


✅ requirements.txt 파일이 생성되었습니다.


In [ ]:
df.to_csv("/content/pypi_typo_analysis5.csv", index=False)
print("✅ 오타 기반 탐지 결과 저장 완료!")

✅ 오타 기반 탐지 결과 저장 완료!


# 피클 파일을 통한 예측

In [ ]:
import pandas as pd
import pickle

# 1️⃣ 피클 파일 로드 (예: 분석 모델 또는 전처리기)
with open("/content/xgboost_model.pkl", "rb") as f:
    model = pickle.load(f)

# 2️⃣ CSV 파일 로드
df = pd.read_csv("/content/pypi_typo_analysis5.csv")

# 3️⃣ 필요한 전처리 (예: summary, version, download 등)
# 예시: summary 길이, 엔트로피, 버전 형식 등
import math
import re
from collections import Counter

def shannon_entropy(s):
    if not s:
        return 0
    prob = [v / len(s) for v in Counter(s).values()]
    return -sum(p * math.log2(p) for p in prob)

def is_valid_version(v):
    return bool(re.match(r"^\d+\.\d+\.\d+$", str(v).strip()))



df["summary"] = df["summary"].fillna("")
df["summary_length"] = df["summary"].apply(len)
df["summary_entropy"] = df["summary"].apply(shannon_entropy)
df["version_valid"] = df["version"].apply(is_valid_version)
df["download"] = df["download"].fillna(0).astype(int)
df["download_log"] = df["download"].apply(lambda x: math.log1p(x))

# 4️⃣ 모델 입력 피처 선택
feature_order = [
    "summary_length", "summary_entropy", "download_log",
    "is_disposable", "summary_too_short", "summary_too_long",
    "summary_low_entropy", "download_too_low", "download_too_high",
    "version_valid"
]

X = df[feature_order]


# 5️⃣ 예측 수행
df["is_malicious"] = model.predict(X)

# 6️⃣ 결과 출력
print(df[["name", "is_malicious"]])


  name  is_malicious
0  pip             0
1   uv             0


#결론 출력하기

In [ ]:
def get_malicious_reasons(row):
    reasons = []
    if row.get("summary_too_short"): reasons.append("요약이 너무 짧음")
    if row.get("summary_too_long"): reasons.append("요약이 너무 김")
    if row.get("summary_low_entropy"): reasons.append("요약이 자동 생성된 것으로 의심됨")
    if row.get("download_too_low"): reasons.append("다운로드 수가 비정상적으로 낮음")
    if row.get("download_too_high"): reasons.append("다운로드 수가 비정상적으로 높음")
    if not row.get("version_valid"): reasons.append("버전 형식이 올바르지 않음")
    if row.get("is_disposable"): reasons.append("일회용 이메일 사용 의심")
    if row.get("is_typo_like"): reasons.append("인기 패키지 이름과 유사한 오타 기반 이름")
    return reasons

def get_normal_reasons(row):
    reasons = []
    if not row.get("summary_too_short"): reasons.append("요약 길이 적절함")
    if not row.get("summary_too_long"): reasons.append("요약이 너무 길지 않음")
    if not row.get("summary_low_entropy"): reasons.append("요약이 사람이 작성한 것으로 보임")
    if not row.get("download_too_low"): reasons.append("다운로드 수가 충분함")
    if not row.get("download_too_high"): reasons.append("다운로드 수가 과도하지 않음")
    if row.get("version_valid"): reasons.append("버전 형식이 올바름")
    if not row.get("is_disposable"): reasons.append("신뢰할 수 있는 이메일 사용")
    if not row.get("is_typo_like"): reasons.append("이름이 인기 패키지와 유사하지 않음")
    return reasons

# 전체 패키지 순회
with open("/content/pypi_malicious_reason_report.txt", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        pkg_name = row.get("name", "unknown")
        label = row.get("is_malicious", 0)

        if label == 1:
            reasons = get_malicious_reasons(row)
            status = "❌ 악성"
        else:
            reasons = get_normal_reasons(row)
            status = "✅ 정상"

        reason_text = " / ".join(reasons) if reasons else "판단 기준 없음"
        f.write(f"{status} 📦 {pkg_name}\n→ 판단 이유: {reason_text}\n\n")

print("📄 악성/정상 판단 이유가 포함된 리포트가 저장되었습니다.")


📄 악성/정상 판단 이유가 포함된 리포트가 저장되었습니다.


In [ ]:
import pkg_resources

# Generate requirements.txt
libraries = [
    "pandas",
    "numpy",
    "math", # math is a built-in module, no version needed
    "re", # re is a built-in module, no version needed
    "collections", # collections is a built-in module, no version needed
    "sklearn", # Use scikit-learn for requirements.txt
    "requests",
    "Levenshtein",
    "google-cloud-bigquery",
    "google-api-core",
    "zipfile", # zipfile is a built-in module, no version needed
    "os", # os is a built-in module, no version needed
    "csv", # csv is a built-in module, no version needed
    "pickle", # pickle is a built-in module, no version needed
    "xgboost" # Assuming xgboost was used for the model
]

with open("requirements.txt", "w") as f:
    for lib in libraries:
        try:
            # For built-in modules, skip writing version
            if lib in ["math", "re", "collections", "zipfile", "os", "csv", "pickle"]:
                f.write(f"{lib}\n")
            elif lib == "sklearn": # Handle scikit-learn
                 f.write(f"scikit-learn=={pkg_resources.get_distribution('scikit-learn').version}\n")
            else:
                f.write(f"{lib}=={pkg_resources.get_distribution(lib).version}\n")
        except pkg_resources.DistributionNotFound:
            f.write(f"{lib} # Version not found\n")


print("✅ requirements.txt 파일이 생성되었습니다.")

✅ requirements.txt 파일이 생성되었습니다.
